In [1]:
import torchvision.transforms as transforms
import cv2
import torch
import numpy as np
from torch import nn
#from google.colab import drive
import torchvision
from torchvision.transforms import ToTensor
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
import torch.nn as nn
from torchvision import transforms, datasets
import shutil
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
#from efficientnet_pytorch import EfficientNet
from collections import defaultdict
from torch.nn import functional as F
import torchvision.models as models
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import _LRScheduler
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import LinearLR
from sklearn.model_selection import train_test_split
import random
import os
#from pytorch_grad_cam import GradCAM
#from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision import transforms
import torch
import cv2
from pathlib import Path

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pickle

!pip install grad-cam

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for grad-cam: filename=grad_cam-1.5.3-py3-none-any.whl size=38657 sha256=b919e99097397d442d060b38b2aadaec6f1a05c4b79b846b29b5e4b4f1d0f773
  Stored in directory: /root/.cache/pip/wheels/2e/ce/70/fe64f851895eae830b3c63ec7fc464cfa7c81aeb7ad4f68063
Successfully built grad-cam


In [2]:
MEAN = torch.tensor((0.485, 0.456, 0.406))
STD  = torch.tensor((0.229, 0.224, 0.225))
# BATCH_SIZE = 64
NUM_CLASSES = 2 # Healthy & Not Healthy
TRAIN_DATA_PATH = "/kaggle/input/binary-coco-first-article/train"
TEST_DATA_PATH = "/kaggle/input/binary-coco-first-article/test"

In [3]:
class EfficientNetB1(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB1, self).__init__()
        self.effnet = models.efficientnet_b1(pretrained=True)
        num_ftrs = self.effnet.classifier[1].in_features
        self.effnet.classifier[1].fc = nn.Linear(num_ftrs, NUM_CLASSES)

    def forward(self, x):
        return self.effnet(x)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [5]:
# Define model and criterion
model = EfficientNetB1(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 121MB/s]


In [6]:
model.load_state_dict(torch.load("/kaggle/input/effnet-updated-unhealthy/pytorch/default/1/effnet_weights (2).pth"))

/tmp/ipykernel_24/260028967.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/effnet-updated-unhealthy/pytorch/default/1/ef

<All keys matched successfully>

In [7]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

def custom_loader(path):
    return datasets.folder.default_loader(path)

In [8]:
# model.eval()
# incorrect_predictions = []
# dataset_directory = Path("/kaggle/input/corner-cases/corner_cases")
# test_dataset = datasets.ImageFolder(root=dataset_directory, transform=TRANSFORM_IMG, loader=custom_loader)

# class_names = test_dataset.classes
# device = 'cuda'
# num_images_to_show = 10
# num_cols = 3
# for i in range(len(test_dataset)):
#     input_data, true_label = test_dataset[i]
#     input_data = input_data.unsqueeze(0).to(device)

#     output = model(input_data)
#     _, predicted_label = torch.max(output, 1)

#     if predicted_label != true_label:
#         incorrect_predictions.append((input_data, true_label, predicted_label.item()))

#     if len(incorrect_predictions) >= num_images_to_show:
#         break

# # num_rows = (len(incorrect_predictions) + num_cols - 1) // num_cols

# # fig, axs = plt.subplots(num_rows, num_cols, figsize=(25, num_rows * 3))

# for i, (input_data, true_label, predicted_label) in enumerate(incorrect_predictions):
#     title_color = 'red'
#     img = input_data.squeeze().cpu().numpy().transpose(1, 2, 0) / 2 + 0.5
#     true_label_name = class_names[true_label]
#     predicted_label_name = class_names[predicted_label]

# #     if num_rows == 1:
# #         axs[i % num_cols].imshow(img)
# #         axs[i % num_cols].set_title(f'Truth: {true_label_name}\nPredicted: {predicted_label_name}', color=title_color)
# #         axs[i % num_cols].axis('off')
# #     else:
# #         row_idx = i // num_cols
# #         col_idx = i % num_cols
# #         axs[row_idx, col_idx].imshow(img)
# #         axs[row_idx, col_idx].set_title(f'Truth: {true_label_name}\nPredicted: {predicted_label_name}', color=title_color)
# #         axs[row_idx, col_idx].axis('off')

# # plt.show()

In [9]:
model.eval()
preprocess = TRANSFORM_IMG

In [10]:
# for i in range(len(test_dataset)):
#     print(test_dataset[i][1])

In [11]:
# for i in range(len(test_dataset)):
#     output = model(test_dataset[i][0].unsqueeze(0).to(device))
#     _, predicted_label = torch.max(output, 1)
#     if int(predicted_label[0]) == 0:
#         label = "healthy"
#     else:
#         label = "unhealthy"
#     print(label)
    

In [12]:
model.effnet.features[-1][0]

Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [13]:
model.effnet.features[7][0].block[-1][0]

Conv2d(1152, 320, kernel_size=(1, 1), stride=(1, 1), bias=False)

# test_2

## Unhealthy

In [14]:
temp_path = "/kaggle/input/test-3/desease(looks_healthy)/"
save_path = "/kaggle/working/gradcam/test-3/desease(looks_healthy)/"

# Unhealthy
original_label = "Unhealthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [15]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

In [16]:
temp_path = "/kaggle/input/test-3/desease(looks_unhealthy)/"
save_path = "/kaggle/working/gradcam/test-3/desease(looks_unhealthy)/"

# Unhealthy
original_label = "Unhealthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [17]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

## Healthy

In [18]:
temp_path = "/kaggle/input/test-3/normal(looks_healthy)/"
save_path = "/kaggle/working/gradcam/test-3/normal(looks_healthy)/"

# Healthy
original_label = "Healthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [19]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

In [20]:
temp_path = "/kaggle/input/test-3/normal(looks_unhealthy)/"
save_path = "/kaggle/working/gradcam/test-3/normal(looks_unhealthy)/"

# Healthy
original_label = "Healthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [21]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

# binary-coco

In [22]:
temp_path = "/kaggle/input/binary-coco-first-article/test/healthy/"
save_path = "/kaggle/working/gradcam/binary-coco-first-article/test/healthy/"

# Healthy
original_label = "Healthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [23]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

In [24]:
temp_path = "/kaggle/input/binary-coco-first-article/test/unhealthy/"
save_path = "/kaggle/working/gradcam/binary-coco-first-article/test/unhealthy/"

# Unhealthy
original_label = "Unhealthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [25]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

# corner-cases

In [26]:
temp_path = "/kaggle/input/corner-cases/corner_cases/healthy/"
save_path = "/kaggle/working/gradcam/corner_cases/healthy/"

# Healthy
original_label = "Healthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [27]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1

In [28]:
temp_path = "/kaggle/input/corner-cases/corner_cases/unhealthy_2/"
save_path = "/kaggle/working/gradcam/corner_cases/unhealthy_2/"

# Unhealthy
original_label = "Unhealthy"
list_of_paths = []
list_of_names = []

for file_name in os.listdir(temp_path):
    list_of_paths.append(temp_path + file_name)
    
list_of_names = [os.path.splitext(f)[0] for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]
os.makedirs(save_path, exist_ok=True)

In [29]:
i = 0
for file_path in list_of_paths:
    image_path = file_path
    img = cv2.imread(image_path)[..., ::-1]

    pil_img = Image.fromarray(img)
    input_tensor = preprocess(pil_img).unsqueeze(0)
    model.to('cuda')

    cam = GradCAM(model=model, target_layers=[model.effnet.features[-1][0]])

    grayscale_cam = cam(input_tensor=input_tensor)
    resized_cam = cv2.resize(grayscale_cam[0], (img.shape[1], img.shape[0]))
    visualization = show_cam_on_image(img / 255.0, resized_cam, use_rgb=True)
    
    output = model(input_tensor.to(device))
    _, predicted = torch.max(output, 1)
    if int(predicted[0]) == 0:
        predicted_label = "Healthy"
    else:
        predicted_label = "Unhealthy"

    fig, ex = plt.subplots(1, 2, dpi = 150)
    ex[0].imshow(visualization)
    ex[0].set_title("Original: " + original_label)
    ex[1].imshow(img)
    ex[1].set_title("Predicted: " + predicted_label)
    
    plt.savefig(fname=(save_path+"gradcam-"+list_of_names[i]+".jpg"))
    plt.close()
    i += 1